# Establishing database connection, running the main query

Grouping fires by cause, creating broader subcategories

In [8]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('../data/FPA_FOD_20170508.sqlite')

query_1 = """
        SELECT
            FIRE_CODE, FIRE_NAME, FIRE_YEAR, datetime(DISCOVERY_DATE) AS DISCOVERY_DATE,
            DISCOVERY_DOY, DISCOVERY_TIME, STAT_CAUSE_DESCR, CASE
             WHEN STAT_CAUSE_DESCR = 'Lightning' THEN 'Natural'
             WHEN STAT_CAUSE_DESCR IN ('Campfire', 'Children', 'Debris Burning', 'Equipment Use', 'Fireworks', 'Powerline', 'Railroad', 'Smoking', 'Structure') 
              THEN 'Accidental Human Activity'
             WHEN STAT_CAUSE_DESCR = 'Arson' THEN 'Intentional Human Activity'
             ELSE 'Miscellaneous/Unknown'
            END AS CAUSE_CATEGORY,
            datetime(CONT_DATE) AS CONT_DATE,
            CONT_DOY, CONT_TIME, FIRE_SIZE, FIRE_SIZE_CLASS, LATITUDE, LONGITUDE, STATE, COUNTY
        FROM
            Fires
        WHERE
            COUNTY IS NOT NULL
            AND CONT_DATE IS NOT NULL
            # AND FIRE_SIZE > 100;
        """

df_1 = pd.read_sql_query(query_1, conn)

df_1.head()

,FIRE_CODE,FIRE_NAME,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_DESCR,CAUSE_CATEGORY,CONT_DATE,CONT_DOY,CONT_TIME,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,COUNTY
0,,POWER,2004,2004-10-06 00:00:00,280,1415,Equipment Use,Accidental Human Activity,2004-10-21 00:00:00,295,1000,16823.0,G,38.523333,-120.211667,CA,5
1,BHA3,FREDS,2004,2004-10-13 00:00:00,287,1618,Equipment Use,Accidental Human Activity,2004-10-17 00:00:00,291,1800,7700.0,G,38.780000,-120.260000,CA,17
2,BKC8,AUSTIN CREEK,2005,2005-02-12 00:00:00,43,1520,Debris Burning,Accidental Human Activity,2005-02-13 00:00:00,44,0330,125.0,D,36.001667,-81.590000,NC,27
3,BEW8,THOMPSON BUTTE,2005,2005-07-16 00:00:00,197,1715,Lightning,Natural,2005-07-17 00:00:00,198,1525,119.0,D,43.899167,-102.954722,SD,103
4,BEW8,CHARLES DRAW,2005,2005-07-16 00:00:00,197,1730,Lightning,Natural,2005-07-16 00:00:00,197,2045,119.0,D,43.892778,-102.948056,SD,103


# Adding a column with full state names to the dataframe, as we'll need this for the choropleth map

In [9]:
state_code_to_name = {
    'CA': 'California', 'TX': 'Texas', 'NY': 'New York', 'FL': 'Florida', 'WA': 'Washington',
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CO': 'Colorado',
    'CT': 'Connecticut', 'DE': 'Delaware', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho',
    'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas', 'KY': 'Kentucky',
    'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland', 'MA': 'Massachusetts', 'MI': 'Michigan',
    'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska',
    'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NC': 'North Carolina',
    'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania',
    'RI': 'Rhode Island', 'SC': 'South Carolina', 'SD': 'South Dakota', 'TN': 'Tennessee',
    'UT': 'Utah', 'VT': 'Vermont', 'VA': 'Virginia', 'WV': 'West Virginia', 'WI': 'Wisconsin',
    'WY': 'Wyoming'
}

df_1['STATE_NAME'] = df_1['STATE'].map(state_code_to_name)

df_1.head()

df_1.count


<bound method DataFrame.count of       FIRE_CODE       FIRE_NAME  FIRE_YEAR       DISCOVERY_DATE  \
0                         POWER       2004  2004-10-06 00:00:00   
1          BHA3           FREDS       2004  2004-10-13 00:00:00   
2          BKC8    AUSTIN CREEK       2005  2005-02-12 00:00:00   
3          BEW8  THOMPSON BUTTE       2005  2005-07-16 00:00:00   
4          BEW8    CHARLES DRAW       2005  2005-07-16 00:00:00   
...         ...             ...        ...                  ...   
18362      None    1-51 WILDCAT       2015  2015-08-01 00:00:00   
18363      None          MUNJAR       2015  2015-10-02 00:00:00   
18364      None         CIENEGA       2015  2015-10-12 00:00:00   
18365      None        MERIDIAN       2015  2015-10-03 00:00:00   
18366      None        RICHVALE       2015  2015-10-13 00:00:00   

       DISCOVERY_DOY DISCOVERY_TIME STAT_CAUSE_DESCR  \
0                280           1415    Equipment Use   
1                287           1618    Equipment U

# Generating JSON file from the resulting dataframe

In [10]:
df_1.to_json("./../../frontend/public/fires_reduced.json", orient = "records", indent = 4)

# Closing the connection to the database

In [5]:
conn.close()